#### The code is used to train our model using a ResNet framework with a selected pre-trained model. 
We start from 
1. data augmentation
2. initiaizing our model and hyperparameters
3. training and validating the results for training and validation data
4. and showing our training log

### Initializing libraries

In [ ]:
import os
import glob

import pandas as pd
import numpy as np

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.io import read_image

import urllib.request
from PIL import Image
import cv2
import matplotlib.pyplot as plt

from google.colab import output
output.enable_custom_widget_manager()
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from datetime import datetime

from google.colab import drive
drive.mount('/content/gdrive')

### Adding number labels for each houseplants

In [ ]:
root_path = '/content/gdrive/MyDrive/Colab Notebooks/Final_data/train'
folders = os.listdir(root_path)
folders.sort()
label_dict = dict()
for id, folder in enumerate(folders):
  label_dict[folder] = id
label_dict

{'Aglaonema_commutatum': 0,
 'Asplenium_nidus': 1,
 'Calathea_orbifolia': 2,
 'Chamaedorea_elegans': 3,
 'Dracaena_reflexa': 4,
 'Dracaena_trifasciata': 5,
 'Dypsis_lutescens': 6,
 'Epipremnum_aureum': 7,
 'Hoya_carnosa': 8,
 'Maranta_leuconeura': 9,
 'Monstera_deliciosa': 10,
 'Nephrolepis_cordifolia': 11,
 'Pachira_aquatica': 12,
 'Peperomia_argyreia': 13,
 'Peperomia_obtusifolia': 14,
 'Philodendron_gloriosum': 15,
 'Rhapis_excelsa': 16,
 'Schefflera_arboricola': 17,
 'Tradescantia_zebrina': 18,
 'Zamioculcas': 19}

In [ ]:
# Generate # of category for applying in neural network
n_class = len(label_dict)
print("n_class", n_class)

n_class 20


### Initialize data paths for training, validation and testing datasets

In [ ]:
path_train = '/content/gdrive/MyDrive/Colab Notebooks/Final_data/train'
path_val = '/content/gdrive/MyDrive/Colab Notebooks/Final_data/val'
path_test = '/content/gdrive/MyDrive/Colab Notebooks/Final_data/test'

### Data pre-processing
need to built __init__, __len__, and __getitem__

Reference: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

\_\_len\_\_: https://pytorch.org/docs/master/data.html#torch.utils.data.Dataset

In [ ]:
class GreenFingerDataset(Dataset):

    def __init__(self, data_path, label_dict, oversample=False, transforms=None):
        # retrieve jpg file path from directory: training/validation/testing
        self.files = sorted(glob.glob(os.path.join(data_path, "*/*.jpg")))
        # convert jpg file path to dataframe and attach true label
        self.df = pd.DataFrame(
            dict(
                cat=[f.split('/')[-2] for f in self.files],
                image_path=self.files
            )
        )
        # copy dataframe
        self.df_oversampled = self.df.copy()
        # process oversample function
        if oversample:
            self.oversample()
        # initialize transform procedure
        self.transforms = transforms
        # initialize true label numbering
        self.label_dict = label_dict

    def oversample(self):
        """
        This functions can be called or 
        initialized automatically when the oversample=True
        """
        # Random sampling til the # of sample matches to the # of largest category
        cats = self.df.cat.drop_duplicates().to_numpy()
        cat_sizes = []

        for c in cats:
            n = len(self.df.query(f"cat == '{c}'"))
            cat_sizes.append(n)

        cat_sizes = np.array(cat_sizes)
        dfs = []
        n_majority = np.max(cat_sizes)

        for i, c in enumerate(cats):
            df_cat = self.df.query(f"cat == '{c}'").sample(frac=n_majority/cat_sizes[i], replace=True)
            dfs.append(df_cat)

        self.df_oversampled = pd.concat(dfs, axis=0).reset_index(drop=True)

    def __len__(self):
        return len(self.df_oversampled)

    def __getitem__(self, idx):
        img_src, cat = self.df_oversampled.iloc[idx][["image_path", "cat"]]
        # What's the difference between torchvision's readimage and PIL's Image.open?
        x = Image.open(img_src)
        # x = read_image(img_src)
        if self.transforms:
            x = self.transforms(x)
        x = x.float()
        if len(x.size()) == 2:
            # when it's multi channels, x.size() shows (Channel, Heigh, Weight)
            # when channel is 1, x.size() shows (Heigh, Weight) w/o channel
            x = x.unsqueeze(0) 
            x = torch.cat((x, x, x), axis=0)
        y = self.label_dict[cat]
        y = torch.Tensor([y]).long()
        
        return x, y

### Initialize our model and optimizer

In [ ]:
# Initialize parameters
n_epoch = 20
lr = 3e-4
batch_size = 128
weight_decay=1e-7
img_size = 256

In [ ]:
# Initialize model and optimizer
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# Change fc's out_feature from 1000 to 20
model.fc = nn.Linear(model.fc.in_features, n_class)
model

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Move the model to GPU device
device = torch.device("cuda:0")
model = model.to(device)


### Define transform procedure

In [ ]:
# Transform for training Dataset
transforms_train = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomAffine(degrees=5, scale=(1.2, 1.2), translate=(0.2, 0.1)),
     transforms.RandomResizedCrop(size=img_size, scale=(0.2, 1.0), ratio=(1.0, 1.0)),
     transforms.RandomHorizontalFlip(),
     transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.2, hue=0.05),
     transforms.Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225])
    ])

transforms_train

Compose(
    ToTensor()
    RandomAffine(degrees=[-5.0, 5.0], translate=(0.2, 0.1), scale=(1.2, 1.2))
    RandomResizedCrop(size=(256, 256), scale=(0.2, 1.0), ratio=(1.0, 1.0), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=[0.9, 1.1], contrast=[0.9, 1.1], saturation=[0.8, 1.2], hue=[-0.05, 0.05])
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [ ]:
# Transform for Validation Dataset
transforms_val = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(img_size),
     transforms.CenterCrop(img_size),  # unify all image as square shape
     transforms.Normalize(             # tied with pre-trained weights
                          mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
     ])

transforms_val

Compose(
    ToTensor()
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
    CenterCrop(size=(256, 256))
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [ ]:
# Transform for Testing Dataset
transforms_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(img_size),
     transforms.CenterCrop(img_size),
     transforms.Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225])
    ])

transforms_test

Compose(
    ToTensor()
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
    CenterCrop(size=(256, 256))
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

### Initialize dataset

In [ ]:
# Training Dataset
ds_train = GreenFingerDataset(
    path_train,
    label_dict,
    oversample=True,
    transforms=transforms_train
)
print("ds_train", len(ds_train))

ds_train 21900


In [ ]:
ds_train.df

,cat,image_path
0,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
1,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
2,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
3,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
4,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
...,...,...
9157,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...
9158,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...
9159,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...
9160,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...


In [ ]:
ds_train.df_oversampled

,cat,image_path
0,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
1,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
2,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
3,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
4,Aglaonema_commutatum,/content/gdrive/MyDrive/Colab Notebooks/Final_...
...,...,...
21895,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...
21896,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...
21897,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...
21898,Zamioculcas,/content/gdrive/MyDrive/Colab Notebooks/Final_...


In [ ]:
ds_train.df_oversampled['cat'].value_counts()

Aglaonema_commutatum      1095
Asplenium_nidus           1095
Tradescantia_zebrina      1095
Schefflera_arboricola     1095
Rhapis_excelsa            1095
Philodendron_gloriosum    1095
Peperomia_obtusifolia     1095
Peperomia_argyreia        1095
Pachira_aquatica          1095
Nephrolepis_cordifolia    1095
Monstera_deliciosa        1095
Maranta_leuconeura        1095
Hoya_carnosa              1095
Epipremnum_aureum         1095
Dypsis_lutescens          1095
Dracaena_trifasciata      1095
Dracaena_reflexa          1095
Chamaedorea_elegans       1095
Calathea_orbifolia        1095
Zamioculcas               1095
Name: cat, dtype: int64

In [ ]:
# Validation Dataset
ds_val = GreenFingerDataset(
    path_val,
    label_dict,
    transforms=transforms_val
)
print("ds_val", len(ds_val))

ds_val 494


In [ ]:
# Testing Dataset
ds_test = GreenFingerDataset(
    path_test,
    label_dict,
    transforms=transforms_test
)
print("ds_test", len(ds_test))

ds_test 449


### Start training!

In [ ]:
# seen counter
seen = 0

# Path for saving the training log
user = 'Zephyr'  # Tom, CY, Zephyr --> Change your name accordingly
log_path = os.path.join('/content/gdrive/MyDrive/Colab Notebooks/Training_Log', user)

In [ ]:
# Initialize Data Loader
dl_train = DataLoader(
    ds_train,
    shuffle=True,
    batch_size=batch_size,
    drop_last=True
)
dl_val = DataLoader(
    ds_val,
    shuffle=False,
    batch_size=batch_size,
    drop_last=False,
)

Save & Load Model

Reference: https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
# Initialize Loss Function
loss_fn = nn.CrossEntropyLoss()

# Initialize Log
train_log = []

# Set the frequency for validatiing the data acc and saving the training log
val_every = 1
save_every = 1

for i in range(n_epoch):
    print(f'--------- This is epoch {(i+1)} ---------')
    model.train()
    # no need to switch the model back to model.train()?
    for (x, y) in tqdm(dl_train, desc='Training', position=0, leave=True):
        x = x.to(device)
        y = y.to(device)
        # what's the difference if we put optimizer.zero_grad() to later step?
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y[:, 0])
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            seen = seen + x.size(0)
            acc_train = torch.sum(torch.argmax(output, axis=1) == y[:, 0]) / np.sum(batch_size)
            print(f' | Seen {seen} | Loss {round(loss.item(), 4)} | Train Acc: {round(acc_train.item(), 3)}')

    # Validate every 1 epochs
    if (i+1) % val_every == 0:
        model.eval()

        predictions = []

        with torch.no_grad():
            for (x, y) in tqdm(dl_val, desc='Validation', position=0, leave=True):
                x = x.to(device)
                y = y.to(device)
                output = model(x)
                predictions.append(output.cpu().numpy())
            # Compute Validation Accuracy
            val_preds = np.concatenate(predictions, axis=0)
            preds = np.argmax(val_preds, axis=1)
            y = np.concatenate([item[1].numpy() for item in ds_val])
            acc_val = np.sum(preds == y) / len(preds)
            
            print(f' | Seen: {seen} | Loss: {round(loss.item(), 4)} | Val Acc: {round(acc_val, 3)}')
            # Append Training Log
            train_log.append(
                dict(ePoch = i+1,
                     seen = seen,
                     loss_train = loss.item(),
                     acc_train = acc_train.item(),
                     acc_val = acc_val.item()
                     ))
    if i == 0 or (i > 0 and acc_val > acc_val_temp):
        torch.save({
          'epoch': i+1,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss.item(), 
          'train_log': train_log
        },
          os.path.join(
              # Select your path from above
              log_path, f'epoch_{i+1}_{datetime.now()}.pt'
          ))
    acc_val_temp = acc_val

--------- This is epoch 1 ---------


Training:   1%|          | 1/171 [00:59<2:49:24, 59.79s/it]

 | Seen 128 | Loss 3.0068 | Train Acc: 0.062


Training:   1%|          | 2/171 [01:51<2:35:37, 55.25s/it]

 | Seen 256 | Loss 2.9774 | Train Acc: 0.109


...

 | Seen 48128 | Loss 0.0311 | Train Acc: 0.992


Training:  20%|██        | 35/171 [06:09<23:45, 10.49s/it]

 | Seen 48256 | Loss 0.1871 | Train Acc: 0.953


Training:  20%|██        | 35/171 [06:17<24:26, 10.79s/it]


FileNotFoundError: ignored

### Show training log

In [ ]:
df_train = pd.DataFrame(train_log)
df_train

In [ ]:
plt.figure()
plt.scatter(df_train["seen"], df_train["acc_train"])
plt.plot(df_train["seen"], df_train["acc_train"], label="training")
plt.plot(df_train["seen"], df_train["acc_val"], label="validation")
plt.legend(edgecolor='white')
plt.ylim(0.5, 1.0)
plt.xlabel("iterations")
plt.ylabel("accuracy")
plt.show()